In [1]:
import torch
import pandas as pd
import numpy as np
import torch.onnx
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
## Models alternatifs

from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

/home/olivier/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('/kaggle/input/amazon-reviews/train.csv', nrows = 10000)

cols = ['polarity', 'recommendations', 'reviews']
df.columns = cols

df['polarity'] = df['polarity'].replace({1: 0, 2: 1})
df.head(5)

,polarity,recommendations,reviews
0,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
1,1,Amazing!,This soundtrack is my favorite music of all ti...
2,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
3,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
4,1,an absolute masterpiece,I am quite sure any of you actually taking the...


In [4]:
df["polarity"].value_counts()

polarity
0    5098
1    4902
Name: count, dtype: int64

## Data Preparation

In [5]:
train_texts = df['reviews'].tolist()
train_labels = df['polarity'].tolist()

input_ids = tokenizer(train_texts, return_tensors='pt', padding=True, truncation=True)['input_ids']
labels = torch.tensor(train_labels)

dataset = TensorDataset(input_ids, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## Model training

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.train()
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

accumulation_steps = 4
num_epochs = 1 #30
correct_predictions = 0
total_predictions = 0

for epoch in range(num_epochs):
    for i, batch in enumerate(dataloader):
        inputs, labels = batch
        attention_mask = (inputs != tokenizer.pad_token_id).float()
        inputs, labels, attention_mask = inputs.to(device), labels.to(device), attention_mask.to(device)
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(predictions == labels).item()
            total_predictions += len(labels)

        if (i + 1) % accumulation_steps == 0 or i == len(dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()
        torch.cuda.empty_cache()

    accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {accuracy}')
    correct_predictions = 0
    total_predictions = 0

KeyboardInterrupt: 

## Prediction using the model 

In [ ]:
df["reviews"][77]

In [ ]:
test_text = df["reviews"][77]

test_input_ids = tokenizer(test_text, return_tensors='pt')['input_ids']
test_input_ids = test_input_ids.to(device)

with torch.no_grad():
    model.eval()
    outputs = model(test_input_ids)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1)
 
    if prediction == 1:
        print("Sentiment positif")
    else:
        print("Sentiment négatif")

In [ ]:
test_text = "This product is not bad at all"

test_input_ids = tokenizer(test_text, return_tensors='pt')['input_ids']
test_input_ids = test_input_ids.to(device)

with torch.no_grad():
    model.eval()
    outputs = model(test_input_ids)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1)
 
    if prediction == 1:
        print("Sentiment positif")
    else:
        print("Sentiment négatif")

In [ ]:
prediction

---------------------------------------------------------------------------
OutOfMemoryError                          Traceback (most recent call last)
Cell In[14], line 19
     16 optimizer.zero_grad()
     18 with torch.set_grad_enabled(True):
---> 19     outputs = model(inputs, attention_mask=attention_mask, labels=labels)
     20     loss = outputs.loss
     21     loss.backward()

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:1208, in XLMRobertaForSequenceClassification.forward(self, input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, output_attentions, output_hidden_states, return_dict)
   1200 r"""
   1201 labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
   1202     Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
   1203     config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
   1204     `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
   1205 """
   1206 return_dict = return_dict if return_dict is not None else self.config.use_return_dict
-> 1208 outputs = self.roberta(
   1209     input_ids,
   1210     attention_mask=attention_mask,
   1211     token_type_ids=token_type_ids,
   1212     position_ids=position_ids,
   1213     head_mask=head_mask,
   1214     inputs_embeds=inputs_embeds,
   1215     output_attentions=output_attentions,
   1216     output_hidden_states=output_hidden_states,
   1217     return_dict=return_dict,
   1218 )
   1219 sequence_output = outputs[0]
   1220 logits = self.classifier(sequence_output)

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:837, in XLMRobertaModel.forward(self, input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, encoder_hidden_states, encoder_attention_mask, past_key_values, use_cache, output_attentions, output_hidden_states, return_dict)
    828 head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)
    830 embedding_output = self.embeddings(
    831     input_ids=input_ids,
    832     position_ids=position_ids,
   (...)
    835     past_key_values_length=past_key_values_length,
    836 )
--> 837 encoder_outputs = self.encoder(
    838     embedding_output,
    839     attention_mask=extended_attention_mask,
    840     head_mask=head_mask,
    841     encoder_hidden_states=encoder_hidden_states,
    842     encoder_attention_mask=encoder_extended_attention_mask,
    843     past_key_values=past_key_values,
    844     use_cache=use_cache,
    845     output_attentions=output_attentions,
    846     output_hidden_states=output_hidden_states,
    847     return_dict=return_dict,
    848 )
    849 sequence_output = encoder_outputs[0]
    850 pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:525, in XLMRobertaEncoder.forward(self, hidden_states, attention_mask, head_mask, encoder_hidden_states, encoder_attention_mask, past_key_values, use_cache, output_attentions, output_hidden_states, return_dict)
    514     layer_outputs = self._gradient_checkpointing_func(
    515         layer_module.__call__,
    516         hidden_states,
   (...)
    522         output_attentions,
    523     )
    524 else:
--> 525     layer_outputs = layer_module(
    526         hidden_states,
    527         attention_mask,
    528         layer_head_mask,
    529         encoder_hidden_states,
    530         encoder_attention_mask,
    531         past_key_value,
    532         output_attentions,
    533     )
    535 hidden_states = layer_outputs[0]
    536 if use_cache:

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:414, in XLMRobertaLayer.forward(self, hidden_states, attention_mask, head_mask, encoder_hidden_states, encoder_attention_mask, past_key_value, output_attentions)
    402 def forward(
    403     self,
    404     hidden_states: torch.Tensor,
   (...)
    411 ) -> Tuple[torch.Tensor]:
    412     # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
    413     self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
--> 414     self_attention_outputs = self.attention(
    415         hidden_states,
    416         attention_mask,
    417         head_mask,
    418         output_attentions=output_attentions,
    419         past_key_value=self_attn_past_key_value,
    420     )
    421     attention_output = self_attention_outputs[0]
    423     # if decoder, the last output is tuple of self-attn cache

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:341, in XLMRobertaAttention.forward(self, hidden_states, attention_mask, head_mask, encoder_hidden_states, encoder_attention_mask, past_key_value, output_attentions)
    331 def forward(
    332     self,
    333     hidden_states: torch.Tensor,
   (...)
    339     output_attentions: Optional[bool] = False,
    340 ) -> Tuple[torch.Tensor]:
--> 341     self_outputs = self.self(
    342         hidden_states,
    343         attention_mask,
    344         head_mask,
    345         encoder_hidden_states,
    346         encoder_attention_mask,
    347         past_key_value,
    348         output_attentions,
    349     )
    350     attention_output = self.output(self_outputs[0], hidden_states)
    351     outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them

File /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

File /opt/conda/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:237, in XLMRobertaSelfAttention.forward(self, hidden_states, attention_mask, head_mask, encoder_hidden_states, encoder_attention_mask, past_key_value, output_attentions)
    234     past_key_value = (key_layer, value_layer)
    236 # Take the dot product between "query" and "key" to get the raw attention scores.
--> 237 attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
    239 if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
    240     query_length, key_length = query_layer.shape[2], key_layer.shape[2]

OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 15.90 GiB total capacity; 14.68 GiB already allocated; 45.75 MiB free; 15.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF## .onnx generation

In [11]:
model.eval()
batch_size = 32
max_sequence_length = 512

onnx_file_path = "/kaggle/working/xlm_r.onnx"
dummy_input = torch.ones(1, max_sequence_length, dtype=torch.long).to(model.device)
dynamic_axes = {'input_ids': {0: 'batch_size', 1: 'sequence_length'}}

torch.onnx.export(model, dummy_input, onnx_file_path, verbose=True, input_names=['input_ids'], output_names=['logits'], dynamic_axes=dynamic_axes)
print(f"ONNX model saved at: {onnx_file_path}")

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX model saved at: /kaggle/working/roberta.onnx
